In [1]:
import pandas as pd
import numpy as np
import os
%load_ext autoreload
%autoreload 2
from src.model.one_class import DeepSVDD
from src.model.reconstruction import DAGMM, MemAutoEncoder
from src.trainer.ids import DeepSVDDIDSTrainer, MemAEIDSTrainer, DAGMMIDSTrainer
from src.datamanager.dataset import IDS2017Dataset
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
dataset_path = "../../data/IDS2017/ids2017.csv"
dataset = IDS2017Dataset(path=dataset_path)
attack_types = np.unique(dataset.labels)
performance_hist = {
        model_name: {
            attack_label: [] for attack_label in attack_types
        } for model_name in ["deepsvdd", "dagmm", "memae"]
    }
performance_summary = {
        model_name: {
            attack_label: {} for attack_label in attack_types
        } for model_name in ["deepsvdd", "dagmm", "memae"]
    }
performance_hist

{'deepsvdd': {'BENIGN': [],
  'Bot': [],
  'DDoS': [],
  'DoS': [],
  'FTP-Patator': [],
  'Heartbleed': [],
  'Infiltration': [],
  'PortScan': [],
  'SSH-Patator': [],
  'Web Attack': []},
 'dagmm': {'BENIGN': [],
  'Bot': [],
  'DDoS': [],
  'DoS': [],
  'FTP-Patator': [],
  'Heartbleed': [],
  'Infiltration': [],
  'PortScan': [],
  'SSH-Patator': [],
  'Web Attack': []},
 'memae': {'BENIGN': [],
  'Bot': [],
  'DDoS': [],
  'DoS': [],
  'FTP-Patator': [],
  'Heartbleed': [],
  'Infiltration': [],
  'PortScan': [],
  'SSH-Patator': [],
  'Web Attack': []}}

In [3]:
for model_name in performance_hist.keys():
    rel_path = "./%s/train" % model_name
    df_fname = "%s_misclassifications.csv" % model_name

    for p in os.listdir(rel_path):
        fname = "{}/{}/{}".format(rel_path, p, df_fname)
        df = pd.read_csv(fname, index_col=0)
        df["Accuracy"] = 1 - df["Misclassified ratio"]
        for atk_label in df.index:
            performance_hist[model_name][atk_label].append(df.loc[atk_label, "Accuracy"])
performance_hist

{'deepsvdd': {'BENIGN': [0.9190539382280833,
   0.8911713925169003,
   0.9292741650196151,
   0.8527483029354773,
   0.8226701772479835,
   0.8996589151106942,
   0.8810286050191223,
   0.9015720176631989,
   0.9163963624651744,
   0.8751115389827645,
   0.8084882265873736,
   0.9076369223512681,
   0.8211178714050128,
   0.845417089796141,
   0.8759167316394075,
   0.9041090345256052,
   0.8750684193760153,
   0.8033939530471483,
   0.8409300107183023,
   0.8303058676104874],
  'Bot': [0.02339776195320442,
   0.2741607324516785,
   0.35707019328585965,
   0.6113936927772126,
   0.6144455747711088,
   0.03001017293997965,
   0.030518819938962438,
   0.044252288911495574,
   0.019328585961342792,
   0.2761953204476093,
   0.029501525940996975,
   0.03306205493387604,
   0.2787385554425229,
   0.010681586978636748,
   0.28942014242115977,
   0.03001017293997965,
   0.05493387589013221,
   0.07121057985757884,
   0.3677517802644964,
   0.041200406917599186],
  'DDoS': [0.8534137330406868,

In [4]:
data = []

for model_name in performance_hist.keys():
    tmp = []
    for atk_label in performance_hist[model_name].keys():
        values = performance_hist[model_name][atk_label]
        mean, std = np.mean(values), np.std(values)
        tmp.append("{:2.2f} ({:2.2f})".format(mean * 100, std * 100))
        performance_summary[model_name][atk_label] = {
            "mean": np.mean(values),
            "std": np.std(values)
        }
    if len(data) == 0:
        data = np.array(tmp).reshape(len(tmp), 1)
    else:
        data = np.concatenate((
            data, np.array(tmp).reshape(len(tmp), 1)
        ), axis=1)
np.save("export/ids2017_performance_summary.npy", performance_summary)
performance_summary

{'deepsvdd': {'BENIGN': {'mean': 0.8700534771122888,
   'std': 0.038312947020495015},
  'Bot': {'mean': 0.17436419125127162, 'std': 0.1918868764354121},
  'DDoS': {'mean': 0.7982823935576089, 'std': 0.10936861506023549},
  'DoS': {'mean': 0.7366111502505363, 'std': 0.062482609722435994},
  'FTP-Patator': {'mean': 0.25442176870748295, 'std': 0.25875727491048234},
  'Heartbleed': {'mean': 0.9363636363636363, 'std': 0.1993792018132846},
  'Infiltration': {'mean': 0.7861111111111112, 'std': 0.13179084626973842},
  'PortScan': {'mean': 0.6578562889322345, 'std': 0.21045724675490252},
  'SSH-Patator': {'mean': 0.19749872816686448, 'std': 0.22759972064550835},
  'Web Attack': {'mean': 0.03885321100917431, 'std': 0.04004986476092105}},
 'dagmm': {'BENIGN': {'mean': 0.4011587294323876, 'std': 0.3563899517901862},
  'Bot': {'mean': 0.741149542217701, 'std': 0.30785579236513316},
  'DDoS': {'mean': 0.47764963640482094, 'std': 0.40231375266849423},
  'DoS': {'mean': 0.5539486182903371, 'std': 0.34

In [5]:
summary_df = pd.DataFrame(data, index=attack_types, columns=["DeepSVDD", "DAGMM", "MemAE"])
summary_df.to_csv("export/ids2017_20_runs_summary.csv")
summary_df.to_latex("export/ids2017_20_runs_summary.tex")
summary_df

,DeepSVDD,DAGMM,MemAE
BENIGN,87.01 (3.83),40.12 (35.64),81.47 (2.40)
Bot,17.44 (19.19),74.11 (30.79),7.25 (7.57)
DDoS,79.83 (10.94),47.76 (40.23),82.20 (9.46)
DoS,73.66 (6.25),55.39 (34.26),84.44 (6.99)
FTP-Patator,25.44 (25.88),63.66 (36.77),37.33 (26.15)
Heartbleed,93.64 (19.94),41.82 (46.39),100.00 (0.00)
Infiltration,78.61 (13.18),49.86 (38.21),89.17 (2.90)
PortScan,65.79 (21.05),94.46 (21.67),39.33 (28.83)
SSH-Patator,19.75 (22.76),62.65 (41.27),9.87 (17.63)
Web Attack,3.89 (4.00),72.40 (41.05),36.55 (32.86)


In [6]:
# need to hardcode DAGMM' results since `metric_values` was not saved during training
# precision, recall, f1
dagmm_results = np.array([
    [0.6689, 0.6502, 0.6594],
    [0.4002, 0.3890, 0.3945],
    [0.6176, 0.6004, 0.6089],
    [0.6558, 0.6375, 0.6465],
    [0.4385, 0.4373, 0.4379],
    [0.5301, 0.5153, 0.5226],
    [0.6568, 0.6384, 0.6475],
    [0.3458, 0.3361, 0.3409],
    [0.6567, 0.6397, 0.6481],
    [0.6769, 0.6580, 0.6673],
    [0.4554, 0.4428, 0.4490],
    [0.3291, 0.3199, 0.3244],
    [0.3739, 0.3634, 0.3686],
    [0.4669, 0.4539, 0.4603],
    [0.6177, 0.6004, 0.6089],
    [0.3663, 0.3560, 0.3611],
    [0.6174, 0.6002, 0.6087],
    [0.6792, 0.6603, 0.6696],
    [0.4418, 0.4295, 0.4356],
    [0.6111, 0.5941, 0.6025]
])
dagmm_results = [
    "{:2.2f} ({:2.2f})".format(
        np.mean(dagmm_results[:, col]), np.std(dagmm_results[:, col])
    ) for col in range(dagmm_results.shape[1])
]
dagmm_results

['0.53 (0.12)', '0.52 (0.12)', '0.52 (0.12)']

In [7]:
deepsvdd_model = DeepSVDD(
    in_features=dataset.in_features,
    n_instances=dataset.n_instances,
    device=device,
    n_layers=4,
    compression_factor=2,
    act_fn="relu"
)
deepsvdd_trainer = DeepSVDDIDSTrainer(
    device=device,
    model=deepsvdd_model,
    batch_size=1024,
    lr=0.001,
    n_epochs=200,
)

memae_model = MemAutoEncoder(
    mem_dim=50,
    latent_dim=1,
    shrink_thres=0.0025,
    n_layers=4,
    compression_factor=2,
    act_fn="relu",
    alpha=2e-4,
    in_features=dataset.in_features,
    n_instances=dataset.n_instances,
    device=device
)
memae_trainer = MemAEIDSTrainer(
    device=device,
    model=memae_model,
    batch_size=1024,
    lr=0.001,
    n_epochs=200
)

overall_performance = {
    "deepsvdd": {
        "precision": 0,
        "recall": 0,
        "f1-score": 0,
        "aupr": 0
    },
    "memae": {
        "precision": 0,
        "recall": 0,
        "f1-score": 0,
        "aupr": 0
    }
}

# need to hardcode DAGMM' results since `metric_values` was not saved during training
data = [
    dagmm_results
]

for model_name, model, trainer in [
    ("deepsvdd", deepsvdd_model, deepsvdd_trainer),
    ("memae", memae_model, memae_trainer),
]:
    rel_path = "./%s/best" % model_name
    P, R, F1 = [], [], []
    for p in os.listdir(rel_path):
        fname = rel_path + "/" + p
        trainer, model = trainer.load_from_file(fname, trainer, model, device)
        metrics = trainer.metric_values
        if metrics["precision"]:
            idx_max = np.argmax(metrics["f1-score"])
            P.append(metrics["precision"][idx_max])
            R.append(metrics["recall"][idx_max])
            F1.append(metrics["f1-score"][idx_max])
    precision = "{:2.2f} ({:2.2f})".format(np.mean(P), np.std(P))
    recall = "{:2.2f} ({:2.2f})".format(np.mean(R), np.std(R))
    f1 = "{:2.2f} ({:2.2f})".format(np.mean(F1), np.std(F1))
    data.append([precision, recall, f1])

In [14]:
overall_results = pd.DataFrame(
    np.array(data).T,
    index=["Precision", "Recall", "F1-Score"],
    columns=["DAGMM", "DeepSVDD", "MemAE"]
)
overall_results.to_csv("export/ids2017_overall_results.csv")
overall_results

,DAGMM,DeepSVDD,MemAE
Precision,0.53 (0.12),0.67 (0.05),0.61 (0.08)
Recall,0.52 (0.12),0.65 (0.05),0.59 (0.08)
F1-Score,0.52 (0.12),0.66 (0.05),0.60 (0.08)
